## Three Methods Consuting Project

You've been hired by a dog food company to try to predict why some batches of their dog food are spoiling much quicker than intended! Unfortunately this Dog Food company hasn't upgraded to the latest machinery, meaning that the amounts of the five preservative chemicals they are using can vary a lot, but which is the chemical that has the strongest effect? The dog food company first mixes up a batch of preservative that contains 4 different preservative chemicals (A,B,C,D) and then is completed with a "filler" chemical. The food scientists beelive one of the A,B,C, or D preservatives is causing the problem, but need your help to figure out which one! Use Machine Learning with RF to find out which parameter had the most predicitive power, thus finding out which chemical causes the early spoiling! So create a model and then find out how you can decide which chemical is the problem!

* Pres_A : Percentage of preservative A in the mix
* Pres_B : Percentage of preservative B in the mix
* Pres_C : Percentage of preservative C in the mix
* Pres_D : Percentage of preservative D in the mix
* Spoiled: Label indicating whether or not the dog food batch was spoiled.

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('consult').getOrCreate()

In [0]:
data = spark.read.csv('dbfs:/FileStore/dog_food.csv', inferSchema=True, header=True)

In [0]:
data.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|      1|
|  5|  6|12.0|  7|      1|
|  6|  2|13.0|  6|      1|
|  4|  2|12.0|  1|      1|
|  4|  2|12.0|  3|      1|
| 10|  3|13.0|  9|      1|
|  8|  5|14.0|  5|      1|
|  5|  8|12.0|  8|      1|
|  6|  5|12.0|  9|      1|
|  3|  3|12.0|  1|      1|
|  9|  8|11.0|  3|      1|
|  1| 10|12.0|  3|      1|
|  1|  5|13.0| 10|      1|
|  2| 10|12.0|  6|      1|
|  1| 10|11.0|  4|      1|
|  5|  3|12.0|  2|      1|
|  4|  9|11.0|  8|      1|
|  5|  1|11.0|  1|      1|
|  4|  9|12.0| 10|      1|
|  5|  8|10.0|  9|      1|
+---+---+----+---+-------+
only showing top 20 rows



In [0]:
data.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: double (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: integer (nullable = true)



In [0]:
from pyspark.sql.functions import col, sum as _sum

In [0]:
# missing values
missing_val = data.select([_sum(col(c).isNull().cast('int')).alias(c) for c in data.columns])
missing_val.show()

+---+---+---+---+-------+
|  A|  B|  C|  D|Spoiled|
+---+---+---+---+-------+
|  0|  0|  0|  0|      0|
+---+---+---+---+-------+



## Format for MLlib

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
data.columns

['A', 'B', 'C', 'D', 'Spoiled']

In [0]:
assembler = VectorAssembler(inputCols=['A', 'B', 'C', 'D'], outputCol='features')

In [0]:
output = assembler.transform(data)

In [0]:
final_data = output.select('Spoiled', 'features')

In [0]:
final_data.show()

+-------+-------------------+
|Spoiled|           features|
+-------+-------------------+
|      1| [4.0,2.0,12.0,3.0]|
|      1| [5.0,6.0,12.0,7.0]|
|      1| [6.0,2.0,13.0,6.0]|
|      1| [4.0,2.0,12.0,1.0]|
|      1| [4.0,2.0,12.0,3.0]|
|      1|[10.0,3.0,13.0,9.0]|
|      1| [8.0,5.0,14.0,5.0]|
|      1| [5.0,8.0,12.0,8.0]|
|      1| [6.0,5.0,12.0,9.0]|
|      1| [3.0,3.0,12.0,1.0]|
|      1| [9.0,8.0,11.0,3.0]|
|      1|[1.0,10.0,12.0,3.0]|
|      1|[1.0,5.0,13.0,10.0]|
|      1|[2.0,10.0,12.0,6.0]|
|      1|[1.0,10.0,11.0,4.0]|
|      1| [5.0,3.0,12.0,2.0]|
|      1| [4.0,9.0,11.0,8.0]|
|      1| [5.0,1.0,11.0,1.0]|
|      1|[4.0,9.0,12.0,10.0]|
|      1| [5.0,8.0,10.0,9.0]|
+-------+-------------------+
only showing top 20 rows



## Train test Split

In [0]:
train, test = final_data.randomSplit([0.7,0.3])

## Create Model

In [0]:
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, GBTClassifier

In [0]:
rfc = RandomForestClassifier(featuresCol='features', labelCol='Spoiled')
dtc = DecisionTreeClassifier(featuresCol='features', labelCol='Spoiled')
gbt = GBTClassifier(featuresCol='features', labelCol='Spoiled')

In [0]:
# training model
rfcModel = rfc.fit(train)
dtcModel = dtc.fit(train)
gbtModel = gbt.fit(train)

In [0]:
# make predictions
rfpredict = rfcModel.transform(test)
dctpredict = dtcModel.transform(test)
gbtpredict = gbtModel.transform(test)

## Evaluations

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
eval = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='Spoiled', metricName='accuracy')

In [0]:
# check accuracy
acc_rf = eval.evaluate(rfpredict)
acc_dct = eval.evaluate(dctpredict)
acc_gbt = eval.evaluate(gbtpredict)

In [0]:
print('Here are the result')
print('*' * 100)
print('Random forest has a accuracy result : {:.2f}%'.format(acc_rf))
print('*' * 100)
print('Single Decison tree has a accuracy result : {:.2f}%'.format(acc_dct))
print('*' * 100)
print('Gradeient Boosting  has a accuracy result : {:.2f}%'.format(acc_gbt))



Here are the result
****************************************************************************************************
Random forest has a accuracy result : 0.99%
****************************************************************************************************
Single Decison tree has a accuracy result : 0.95%
****************************************************************************************************
Gradeient Boosting  has a accuracy result : 0.96%


In [0]:
rfcModel.featureImportances

SparseVector(4, {0: 0.0279, 1: 0.031, 2: 0.9173, 3: 0.0238})

## Training Full Data

In [0]:
full_data_rf = rfc.fit(final_data)

In [0]:
full_data_rf.featureImportances

SparseVector(4, {0: 0.0189, 1: 0.0141, 2: 0.9384, 3: 0.0287})

Bingo! Feature at index 2 (Chemical C) is by far the most important feature, meaning it is causing the early spoilage! This is a pretty interesting use of a machine learning model in an alternative way!

## God Job..!